# Tool use

*Last updated: 2024-01-09*

An exciting use case for LLMs is building natural language interfaces for other "tools" (whether those are APIs, functions, databases, etc). LangChain is great for building such interfaces because it has:

- Good model output parsing, which is necessary for returning structured information to pass into a tool
- Large collection of built-in tools
- Provides a lot of flexibility in how you call these tools

In this guide, we will go over two main ways to call tools: chains and agents.

## Setup

Let's first install all the packages needed for this guide:

In [ ]:
%pip install --upgrade --quiet langchain langchain-openai

And set required environment variables:

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# If you'd like to use LangSmith, uncomment the below
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Create a tool

First, we need to create a tool to call. For this example, we will create a custom tool from a function. For more information on all details related to creating custom tools, please see [this guide](/docs/modules/agents/tools/).

In [1]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [2]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(first_int: int, second_int: int) -> int - Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [3]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

## Chains

If we know that we only need to use a tool a fixed number of times, we can create a chain for doing so. Let's create a simple chain that just multiplies user-specified numbers.

### Function calling
One of the most reliable ways to use tools with LLMs is with function calling. This only works with models that explicitly support function calling, like OpenAI models.

For this example we'll use an OpenAI chat model.

In [24]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

Next we define our OpenAI functions. `langchain` comes with utilities for converting any `langchain` Tool into an OpenAI function:

In [25]:
from langchain.tools.render import format_tool_to_openai_function

functions = [format_tool_to_openai_function(multiply)]

In [26]:
functions

[{'name': 'multiply',
  'description': 'multiply(first_int: int, second_int: int) -> int - Multiply two integers together.',
  'parameters': {'title': 'multiplySchemaSchema',
   'type': 'object',
   'properties': {'first_int': {'title': 'First Int', 'type': 'integer'},
    'second_int': {'title': 'Second Int', 'type': 'integer'}},
   'required': ['first_int', 'second_int']}}]

Now we'll bind our functions to our model, meaning the functions will be passed in as part of the payload to the model each time it is invoked. In this case we'll also bind `function_call`, which will force the OpenAI model to always return inputs for the `multiply` function.

In [27]:
model_with_functions = model.bind(
    functions=functions, function_call={"name": "multiply"}
)

We'll chain our model together with a `JsonOutputFunctionsParser`, which returns just the `functions` part of the model output (which is in JSON) as a dictionary. We'll specify `args_only=True` in this case so that only the function arguments and not the function name is returned, since we know which function the arguments are for:

In [28]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = model_with_functions | JsonOutputFunctionsParser(args_only=True)
chain.invoke("what is thirteen times 4")

{'first_int': 13, 'second_int': 4}

If we specified `args_only=False`, our chain output would look like:

In [30]:
(model_with_functions | JsonOutputFunctionsParser(args_only=False)).invoke(
    "what is thirteen times 4"
)

{'arguments': {'first_int': 13, 'second_int': 4}, 'name': 'multiply'}

Suppose we wanted to add some additional instructions to the model on each call. We can do this by including a prompt at the beginning of our chain:

In [31]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Always call function `multiply` with the smaller number passed in first.",
        ),
        ("user", "{input}"),
    ]
)

chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [32]:
chain.invoke({"input": "what is thirteen times 4"})

{'first_int': 4, 'second_int': 13}

And if we want our chain to actually call the tool once the model had determined the tool inputs, we can easily add that to our chain as well:

In [33]:
chain_with_tool = chain | multiply

In [34]:
chain_with_tool.invoke({"input": "what is thirteen times 4"})

52

### Without function calling

There's plenty of models that do not support function calling. If we want to use one of these, we'll need to do a bit more manual prompting to get our model to return structured outputs containing tool inputs.



In [57]:
import re

from langchain.tools.render import render_text_description_and_args

rendered_tools = render_text_description([multiply])
rendered_tools

'multiply: multiply(first_int: int, second_int: int) -> int - Multiply two integers together.'

In [78]:
system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [79]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'first_int': 13, 'second_int': 4}}

In [80]:
from operator import itemgetter

chain = prompt | model | JsonOutputParser() | itemgetter("arguments") | multiply
chain.invoke({"input": "what's thirteen times 4"})

52

### Multiple tools

Suppose we have multiple tools we want the chain to be able to choose from:

In [81]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

With function calling, we can do this like so:

In [82]:
tools = [multiply, add, exponentiate]
model_with_functions = model.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

chain = model_with_functions | JsonOutputFunctionsParser(args_only=False)
chain.invoke("what 17 cubed")

{'arguments': {'base': 17, 'exponent': 3}, 'name': 'exponentiate'}

If we want to run the model selected tool, we can do so using a function that returns the tool based on the model output. Specifically, our function will action return it's own subchain that gets the "arguments" part of the model output and passes it to the chosen tool:

In [83]:
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool


chain = model_with_functions | JsonOutputFunctionsParser(args_only=False) | tool_chain
chain.invoke("what's 17 cubed")

4913

Without function calling we could do something like this:

In [84]:
rendered_tools = render_text_description(tools)
system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

chain = prompt | model | JsonOutputParser() | tool_chain
chain.invoke({"input": "what's 3 plus 1132"})

1135

## Agents

Chains are great when we know the specific sequence of tool usage needed for any user input. But for certain use cases, how many times we use tools depends on the input. In these cases, we want to let the model itself decide how many times to use tools and in what order. [Agents](/docs/modules/agents/) let us do just this.

LangChain comes with a number of built-in agents that are optimized for different use cases. Read about all the [agent types here](/docs/modules/agents/agent_types/).

As an example, let's try out the OpenAI tools agent, which makes use of the new OpenAI tool-calling API (this is only available in the latest OpenAI models, and differs from function-calling in that the model can return multiple function invocations at once):

In [86]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [88]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [90]:
# Choose the LLM that will drive the agent
# Only certain models support this
model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, tools, prompt)

In [91]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [93]:
agent_executor.invoke(
    {"input": "What's 39 multiplied by seven, all raised to the power of 4"}
)



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'first_int': 39, 'second_int': 7}`


273
Invoking: `exponentiate` with `{'base': 273, 'exponent': 4}`


555457184139 multiplied by 7 is 273, and 273 raised to the power of 4 is 5,554,571,841.

> Finished chain.


{'input': "What's 39 multiplied by seven, all raised to the power of 4",
 'output': '39 multiplied by 7 is 273, and 273 raised to the power of 4 is 5,554,571,841.'}

We can see that in this case our agent used two tools — first it multiplied, and then it used the output of that multiplication to exponentiate. With an agent, we can ask questions that require arbitrarily-many uses of our tools:

In [95]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


3645
Invoking: `exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025The result of raising 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The result of raising 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.'}